## 1. Load the dataset

The dataset used in this example is [fine-food reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews) from Amazon. The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of this dataset, consisting of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text).

We will combine the review summary and review text into a single combined text. The model will encode this combined text and it will output a single vector embedding.

To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [6]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.8 MB/s eta 0:00:00


In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00


In [8]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.3 MB/s eta 0:00:00


In [9]:
# imports
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding


In [10]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


# Download Dataset from Kaggle

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [22]:
#goto link https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
#copy api command from thee dots

In [19]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

 93% 226M/242M [00:01<00:00, 157MB/s]
100% 242M/242M [00:01<00:00, 145MB/s]


In [20]:
!unzip /content/amazon-fine-food-reviews.zip

Archive:  /content/amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


# load & inspect dataset

In [21]:
input_datapath = "data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
input_datapath ='/content/Reviews.csv'
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)


,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [23]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.sort_values("Time").tail(top_n * 1)  # first cut to first 1k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)


<ipython-input-23-3eea48aa143b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("Time", axis=1, inplace=True)


1000

## 2. Get embeddings and save them for future reuse

In [ ]:
# get openai-api
import configparser

config = configparser.ConfigParser()
config.read('/content/drive/MyDrive/openapi.txt')

key  = config.get('global','OPENAI_API_KEY')

In [27]:
import openai
openai.api_key = key

In [ ]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))



In [29]:
# save loation
df.to_csv("fine_food_reviews_with_embeddings_1k.csv")


In [73]:
file= '/content/drive/MyDrive/DS-Data/fine_food_reviews_with_embeddings_1k.csv'
df= pd.read_csv(file)

In [ ]:
#df['embedding'].shape
df

In [34]:
import numpy as np

In [39]:
emd = np.array(df['embedding'])

In [40]:
emd.shape

(1000,)

In [44]:
emd[0]

'[0.007079250644892454, -0.027231059968471527, 0.010618875734508038, -0.014612123370170593, 0.004364392254501581, 0.019904378801584244, 0.0006812919746153057, -0.022144995629787445, -0.01928580366075039, -0.013588037341833115, 0.01807614415884018, 0.004106652922928333, -0.032935697585344315, 0.00014250857930164784, 0.01578054390847683, 0.019368279725313187, 0.01786995306611061, -0.026310069486498833, -0.01940951868891716, -0.025100411847233772, -0.04734162986278534, -0.007209838833659887, 0.023780783638358116, -0.00843324325978756, -0.008962469175457954, 0.011416151188313961, 0.029499169439077377, -0.022502394393086433, -0.01014463510364294, 0.021663881838321686, 0.0011254634009674191, 0.009031199850142002, -0.01333373412489891, -0.0174163319170475, -0.005910830572247505, -0.01094190962612629, -0.026475023478269577, 0.00736104603856802, 0.022502394393086433, -0.020275523886084557, 0.019120849668979645, -0.015299429185688496, 0.009216772392392159, -0.008247670717537403, 0.00169077247846

In [63]:
emd = df['embedding'][:5].tolist()
type(emd)

list

In [65]:
# Convert each string representation of a list to a Python list of floats
import ast
embedding_lists = [ast.literal_eval(s) for s in emd]

# Convert the list of lists to a numpy array of floats
embedding_array = np.array(embedding_lists, dtype=np.float32)

In [66]:
embedding_array.shape

(5, 1536)

# text-embedding-ada-002 returns 1536 dim vector

In [68]:
embedding_array[0]

array([ 0.00707925, -0.02723106,  0.01061888, ..., -0.00700708,
       -0.02184258, -0.0375269 ], dtype=float32)

In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib

In [76]:
df1 = pd.DataFrame(df['embedding'])

In [79]:
matrix = df1.embedding.apply(eval).to_list()

In [ ]:
matrix

In [82]:
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib

matrix = df1.embedding.apply(eval).to_list()

# Create a t-SNE model and transform the data
tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
vis_dims = tsne.fit_transform(matrix)

colors = ["red", "darkorange", "gold", "turquiose", "darkgreen"]
x = [x for x,y in vis_dims]
y = [y for x,y in vis_dims]
color_indices = df.Score.values - 1

colormap = matplotlib.colors.ListedColormap(colors)
plt.scatter(x, y, c=color_indices, cmap=colormap, alpha=0.3)
plt.title("Amazon ratings visualized in language using t-SNE")

AttributeError: ignored